# PE files sorter v1.2

In [2]:
%%time
import pefile
import os
import array
import sys
import argparse
import os
import shutil
import struct
from os import path
import glob
import time
import sys
import hashlib
from collections import defaultdict

IMAGE_FILE_MACHINE_I386=332
IMAGE_FILE_MACHINE_IA64=512
IMAGE_FILE_MACHINE_AMD64=34404

class bcolors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    RED = '\033[91m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    
def chunk_reader(fobj, chunk_size=1024):
    """Generator that reads a file in chunks of bytes"""
    while True:
        chunk = fobj.read(chunk_size)
        if not chunk:
            return
        yield chunk

def md5(fname):
    hash_md5 = hashlib.md5()
    with open(fname, 'rb') as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()


def remove_duplicates(path):
    """relies on the md5 function above to remove duplicate files"""
    if not os.path.isdir(path):  # make sure the given directory exists
        print('specified directory does not exist!')
        return
    
    md5_dict = defaultdict(list)
    for root, dirs, files in os.walk(path):  # the os.walk function allows checking subdirectories too...
        for filename in files:
            try:
                filepath = os.path.join(root, filename)
                print(bcolors.GREEN + "Working: %s." % filepath + bcolors.ENDC) 
                file_md5 = md5(filepath)
                md5_dict[file_md5].append(filepath)
            except:
                print (bcolors.RED + "Working Error" + bcolors.ENDC)
    for key in md5_dict:
        try:
            file_list = md5_dict[key]
            while len(file_list) > 1:
                item = file_list.pop()
                print(bcolors.BLUE + "Duplicate: %s. (removing it...)" % item + bcolors.ENDC)    
                os.remove(item)
        except:
                print (bcolors.RED + "Removing Error" + bcolors.ENDC)
    
        
    print('Done!')


def ChehckIsPEObject(filename):
    """check if object is pe or not"""
    # check if the input file exists
    if not os.path.exists(filename):
        parser.error("cannot access input file '%s'" % args[0])
        return False

    # check if the input file is executable
    if not os.path.isfile(filename):
        print ('The given arg is not a file.')
        return False
    else:
        try:
            pe = pefile.PE(filename)
            if not (pe.is_exe() or pe.is_dll()):
                print ('Input file should be executable (PE format: exe or dll)')
                return False
        except pefile.PEFormatError as e:
            print("[-] PEFormatError: %s" % e.value)
            return False

    return True 

def GetBinaryType(filepath):
    """retrieve architecture of pe file 86\64"""
    f=open(filepath, "rb")
    machine = 0
    s=f.read(2)
    f.seek(60)
    s=f.read(4)
    header_offset=struct.unpack("<L", s)[0]
    f.seek(header_offset+4)
    s=f.read(2)
    machine=struct.unpack("<H", s)[0]
    f.close()
    return machine


if __name__ == '__main__':
    
    # Classify set legitimate \ malware
    legitimate = True
    
    # Copy not pe objects to not_pe \ not_pe_clean dirs
    copy_not_pe = False 
    
    # Clean dublicate files matched by md5 hash
    clean_dublicates = False
    
    if legitimate == True:
        root_dir = 'Z:/training_set/not_classified_clean/'
        not_pe_dir = 'Z:/training_set/not_pe_clean/'
        net_pe_dir = 'Z:/training_set/net_pe_clean/'
        x32_dll_dir = 'Z:/training_set/x32_dll_clean/'
        x64_dll_dir = 'Z:/training_set/x64_dll_clean/'
        x32_exe_dir = 'Z:/training_set/x32_exe_clean/'
        x64_exe_dir = 'Z:/training_set/x64_exe_clean/'
    else:
        root_dir = 'Z:/training_set/VirusSignList_Free_140817/'
        not_pe_dir = 'Z:/training_set/not_pe/'
        net_pe_dir = 'Z:/training_set/net_pe/'
        x32_dll_dir = 'Z:/training_set/x32_dll/'
        x64_dll_dir = 'Z:/training_set/x64_dll/'
        x32_exe_dir = 'Z:/training_set/x32_exe/'
        x64_exe_dir = 'Z:/training_set/x64_exe/'
        
    # Remove dublicate files
    if clean_dublicates == True:
        remove_duplicates(root_dir)
        exit()
    
    # scan all files from selected dir
    x = [entry for entry in os.scandir(root_dir+'.') if entry.is_file()]
    i = 0
    
    for f in x:
        print("%d. %s" % (i, f))
        
        is_dotnet = False
                
        # check if valid PE file
        if ChehckIsPEObject(f):
            
            # load pe file to pelib
            try:
                pe = pefile.PE(f)
            except:
                print("[-] pefile open file error: %s")
            
            # check if PE is dotnet and move to net_pe dir
            try:
                isDotNet = pe.OPTIONAL_HEADER.DATA_DIRECTORY[14]
                if isDotNet.VirtualAddress != 0 and isDotNet.Size != 0:
                    print (bcolors.BLUE + "%s PE is .NET executable" % (os.path.abspath(f)) + bcolors.ENDC)
                    shutil.copy(f, net_pe_dir+os.path.basename(f))
                    is_dotnet = True
            except:
                    print (bcolors.RED + "%s error")
            
            # check if PE is x32 EXE OR DLL and move to dir
            if GetBinaryType(f) == IMAGE_FILE_MACHINE_I386:
                
                if(pe.is_exe() and is_dotnet == False):
                    try:
                        print (bcolors.GREEN + "%s PE is x32 EXE" % (os.path.abspath(f)) + bcolors.ENDC)
                        shutil.copy(f, x32_exe_dir+os.path.basename(f))
                    except:
                        print (bcolors.RED + "Error" + bcolors.ENDC)
                    
                # if PE is x32 DLL
                elif(pe.is_dll() and is_dotnet == False):
                    try:
                        print (bcolors.GREEN + "%s PE is x32 DLL" % (os.path.abspath(f)) + bcolors.ENDC)
                        shutil.copy(f, x32_dll_dir+os.path.basename(f))
                    except:
                        print (bcolors.RED + "Error" + bcolors.ENDC)
            
          
            # check if PE is x64 EXE OR DLL and move to dir        
            elif GetBinaryType(f) == IMAGE_FILE_MACHINE_AMD64:
                if(pe.is_exe() and is_dotnet == False):
                    try:
                        print (bcolors.GREEN + "%s PE is x64 EXE" % (os.path.abspath(f))+ bcolors.ENDC)
                        shutil.copy(f, x64_exe_dir+os.path.basename(f))
                    except:
                        print (bcolors.RED + "Error" + bcolors.ENDC)
                    
                # if PE is x64 DLL    
                elif(pe.is_dll() and is_dotnet == False):
                    try:
                        print (bcolors.GREEN + "%s PE is x64 DLL" % (os.path.abspath(f))+ bcolors.ENDC)
                        shutil.copy(f, x64_dll_dir+os.path.basename(f))
                    except:
                        print (bcolors.RED + "Error" + bcolors.ENDC)
                        
            pe.close()
                                                        
        # check if PE is NOT EXE OR DLL and move to dir           
        else:
            print (bcolors.RED + "%s is NOT a PE file" % (os.path.abspath(f))+ bcolors.ENDC)
            if copy_not_pe == True:
                try:
                    shutil.copy2(f, not_pe_dir+os.path.basename(f))
                except:
                    print (bcolors.RED + "Error" + bcolors.ENDC)
            
        i = i + 1
            
               
print (colors.BLUE + "DONE.")        

0. <DirEntry '16Edit.DLL'>
Z:\training_set\not_classified_clean\16Edit.DLL PE is x32 DLL
1. <DirEntry '16Edit.exe'>
Z:\training_set\not_classified_clean\16Edit.exe PE is x32 EXE
2. <DirEntry '2n86ekn.exe'>
Z:\training_set\not_classified_clean\2n86ekn.exe PE is x32 EXE
3. <DirEntry '3.5.3_44358.exe'>
Z:\training_set\not_classified_clean\3.5.3_44358.exe PE is x32 EXE
4. <DirEntry '3.5.3_44396.exe'>
Z:\training_set\not_classified_clean\3.5.3_44396.exe PE is x32 EXE
5. <DirEntry '3.5.4_44498.exe'>
Z:\training_set\not_classified_clean\3.5.4_44498.exe PE is x32 EXE
6. <DirEntry '3.5.4_44632.exe'>
Z:\training_set\not_classified_clean\3.5.4_44632.exe PE is x32 EXE
7. <DirEntry '3.5.5_44994.exe'>
Z:\training_set\not_classified_clean\3.5.5_44994.exe PE is x32 EXE
8. <DirEntry '3.5.5_45271.exe'>
Z:\training_set\not_classified_clean\3.5.5_45271.exe PE is x32 EXE
9. <DirEntry '360tray.exe'>
Z:\training_set\not_classified_clean\360tray.exe PE is x32 EXE
10. <DirEntry '3dframes.exe'>
Z:\training_set\

Z:\training_set\not_classified_clean\accessibilitycpl.dll PE is x32 DLL
79. <DirEntry 'AccessibleMarshal (2).dll'>
Z:\training_set\not_classified_clean\AccessibleMarshal (2).dll PE is x32 DLL
80. <DirEntry 'AccessibleMarshal (3).dll'>
Z:\training_set\not_classified_clean\AccessibleMarshal (3).dll PE is x32 DLL
81. <DirEntry 'AccessibleMarshal (5).dll'>
Z:\training_set\not_classified_clean\AccessibleMarshal (5).dll PE is x32 DLL
82. <DirEntry 'AccessibleMarshal.dll'>
Z:\training_set\not_classified_clean\AccessibleMarshal.dll PE is x32 DLL
83. <DirEntry 'accevent (2).exe'>
Z:\training_set\not_classified_clean\accevent (2).exe PE is x32 EXE
84. <DirEntry 'accevent (3).exe'>
85. <DirEntry 'accevent (4).exe'>
Z:\training_set\not_classified_clean\accevent (4).exe PE is x32 EXE
86. <DirEntry 'accevent (5).exe'>
87. <DirEntry 'accevent (6).exe'>
Z:\training_set\not_classified_clean\accevent (6).exe PE is x64 EXE
88. <DirEntry 'AccEvent (7).exe'>
Z:\training_set\not_classified_clean\AccEvent (7

144. <DirEntry 'AddInProcess (14).exe'>
Z:\training_set\not_classified_clean\AddInProcess (14).exe PE is .NET executable
145. <DirEntry 'AddInProcess (17).exe'>
Z:\training_set\not_classified_clean\AddInProcess (17).exe PE is .NET executable
146. <DirEntry 'AddInProcess (19).exe'>
Z:\training_set\not_classified_clean\AddInProcess (19).exe PE is .NET executable
147. <DirEntry 'AddInProcess (2).exe'>
Z:\training_set\not_classified_clean\AddInProcess (2).exe PE is .NET executable
148. <DirEntry 'AddInProcess (21).exe'>
Z:\training_set\not_classified_clean\AddInProcess (21).exe PE is .NET executable
149. <DirEntry 'AddInProcess (23).exe'>
Z:\training_set\not_classified_clean\AddInProcess (23).exe PE is .NET executable
150. <DirEntry 'AddInProcess (4).exe'>
Z:\training_set\not_classified_clean\AddInProcess (4).exe PE is .NET executable
151. <DirEntry 'AddInProcess (5).exe'>
Z:\training_set\not_classified_clean\AddInProcess (5).exe PE is .NET executable
152. <DirEntry 'AddInProcess (8).exe'>

Z:\training_set\not_classified_clean\AdobeApplicationManager(URIHandler).exe PE is x32 EXE
210. <DirEntry 'AdobeApplicationManager.exe'>
Z:\training_set\not_classified_clean\AdobeApplicationManager.exe PE is x32 EXE
211. <DirEntry 'AdobeCP15.dll'>
Z:\training_set\not_classified_clean\AdobeCP15.dll PE is x32 DLL
212. <DirEntry 'AdobeGCClient.exe'>
Z:\training_set\not_classified_clean\AdobeGCClient.exe PE is x32 EXE
213. <DirEntry 'AdobeIPCBroker.exe'>
Z:\training_set\not_classified_clean\AdobeIPCBroker.exe PE is x32 EXE
214. <DirEntry 'AdobeIPCBrokerCustomHook.exe'>
Z:\training_set\not_classified_clean\AdobeIPCBrokerCustomHook.exe PE is x32 EXE
215. <DirEntry 'AdobeTool.exe'>
Z:\training_set\not_classified_clean\AdobeTool.exe PE is x32 EXE
216. <DirEntry 'Adobe_Helperx32 (2).exe'>
Z:\training_set\not_classified_clean\Adobe_Helperx32 (2).exe PE is x32 EXE
217. <DirEntry 'Adobe_Helperx32.exe'>
Z:\training_set\not_classified_clean\Adobe_Helperx32.exe PE is x32 EXE
218. <DirEntry 'Adobe_Hel

Z:\training_set\not_classified_clean\afcdpsrv.exe PE is x32 EXE
285. <DirEntry 'Aga.Controls.dll'>
Z:\training_set\not_classified_clean\Aga.Controls.dll PE is .NET executable
286. <DirEntry 'agcore.debug.dll'>
Z:\training_set\not_classified_clean\agcore.debug.dll PE is x32 DLL
287. <DirEntry 'agcore.dll'>
Z:\training_set\not_classified_clean\agcore.dll PE is x32 DLL
288. <DirEntry 'agcp (2).exe'>
Z:\training_set\not_classified_clean\agcp (2).exe PE is x32 EXE
289. <DirEntry 'AgentService.exe'>
Z:\training_set\not_classified_clean\AgentService.exe PE is x64 EXE
290. <DirEntry 'agshelper.exe'>
Z:\training_set\not_classified_clean\agshelper.exe PE is x32 EXE
291. <DirEntry 'AGSService.exe'>
Z:\training_set\not_classified_clean\AGSService.exe PE is x32 EXE
292. <DirEntry 'ahadmin.dll'>
Z:\training_set\not_classified_clean\ahadmin.dll PE is x32 DLL
293. <DirEntry 'AhResStd.dll'>
Z:\training_set\not_classified_clean\AhResStd.dll PE is x32 DLL
294. <DirEntry 'AIM (2).dll'>
Z:\training_set\not

...

Z:\training_set\not_classified_clean\wstraceutil (4).exe PE is x32 EXE
21674. <DirEntry 'wstraceutil (6).exe'>
Z:\training_set\not_classified_clean\wstraceutil (6).exe PE is x64 EXE
21675. <DirEntry 'wstraceutil.exe'>
Z:\training_set\not_classified_clean\wstraceutil.exe PE is x64 EXE
21676. <DirEntry 'wstraceutilresources (2).dll'>
Z:\training_set\not_classified_clean\wstraceutilresources (2).dll PE is x32 DLL
21677. <DirEntry 'wstraceutilresources (3).dll'>
21678. <DirEntry 'wstraceutilresources.dll'>
Z:\training_set\not_classified_clean\wstraceutilresources.dll PE is x64 DLL
21679. <DirEntry 'WsUpgrade.dll'>
Z:\training_set\not_classified_clean\WsUpgrade.dll PE is x32 DLL
21680. <DirEntry 'wsutil (10).exe'>
Z:\training_set\not_classified_clean\wsutil (10).exe PE is .NET executable
21681. <DirEntry 'Wsutil (11).exe'>
Z:\training_set\not_classified_clean\Wsutil (11).exe PE is .NET executable
21682. <DirEntry 'Wsutil (12).exe'>
Z:\training_set\not_classified_clean\Wsutil (12).exe PE is 

Z:\training_set\not_classified_clean\wups.dll PE is x32 DLL
21744. <DirEntry 'wups2.dll'>
Z:\training_set\not_classified_clean\wups2.dll PE is x32 DLL
21745. <DirEntry 'wusa (10).exe'>
Z:\training_set\not_classified_clean\wusa (10).exe PE is x32 EXE
21746. <DirEntry 'wusa (12).exe'>
Z:\training_set\not_classified_clean\wusa (12).exe PE is x32 EXE
21747. <DirEntry 'wusa (14).exe'>
Z:\training_set\not_classified_clean\wusa (14).exe PE is x64 EXE
21748. <DirEntry 'wusa (2).exe'>
Z:\training_set\not_classified_clean\wusa (2).exe PE is x32 EXE
21749. <DirEntry 'wusa (3).exe'>
Z:\training_set\not_classified_clean\wusa (3).exe PE is x64 EXE
21750. <DirEntry 'wusa (5).exe'>
Z:\training_set\not_classified_clean\wusa (5).exe PE is x32 EXE
21751. <DirEntry 'wusa (7).exe'>
Z:\training_set\not_classified_clean\wusa (7).exe PE is x64 EXE
21752. <DirEntry 'wusa (9).exe'>
Z:\training_set\not_classified_clean\wusa (9).exe PE is x32 EXE
21753. <DirEntry 'wusa.exe'>
Z:\training_set\not_classified_clean\w

Z:\training_set\not_classified_clean\x86_microsoft-windows-ndis_31bf3856ad364e35_6.1.7600.16385_none_a79d81ea7d62a289_ndismigplugin.dll_7aadce98 PE is x32 DLL
21811. <DirEntry 'x86_microsoft-windows-networkbridge_31bf3856ad364e35_6.1.7600.16385_none_07c046fe67692e98_bridgemigplugin.dll_4c0b8021'>
Z:\training_set\not_classified_clean\x86_microsoft-windows-networkbridge_31bf3856ad364e35_6.1.7600.16385_none_07c046fe67692e98_bridgemigplugin.dll_4c0b8021 PE is x32 DLL
21812. <DirEntry 'x86_microsoft-windows-offlinefiles-core_31bf3856ad364e35_6.1.7600.16385_none_9e1e9f0abd3adf87_cscmig.dll_0a75eb56'>
Z:\training_set\not_classified_clean\x86_microsoft-windows-offlinefiles-core_31bf3856ad364e35_6.1.7600.16385_none_9e1e9f0abd3adf87_cscmig.dll_0a75eb56 PE is x32 DLL
21813. <DirEntry 'x86_microsoft-windows-rasserver_31bf3856ad364e35_6.1.7600.16385_none_ab7f8f7ac9207680_rasmigplugin.dll_7ee2aa40'>
Z:\training_set\not_classified_clean\x86_microsoft-windows-rasserver_31bf3856ad364e35_6.1.7600.16385_

21862. <DirEntry 'xdump.dll'>
Z:\training_set\not_classified_clean\xdump.dll PE is x32 DLL
21863. <DirEntry 'xe (2).dll'>
Z:\training_set\not_classified_clean\xe (2).dll PE is x32 DLL
21864. <DirEntry 'xe.dll'>
Z:\training_set\not_classified_clean\xe.dll PE is x64 DLL
21865. <DirEntry 'XEDParse (2).dll'>
Z:\training_set\not_classified_clean\XEDParse (2).dll PE is x64 DLL
21866. <DirEntry 'XEDParse.dll'>
Z:\training_set\not_classified_clean\XEDParse.dll PE is x32 DLL
21867. <DirEntry 'xerces-c_3_1.dll'>
Z:\training_set\not_classified_clean\xerces-c_3_1.dll PE is x64 DLL
21868. <DirEntry 'xf-adobecc2014.exe'>
Z:\training_set\not_classified_clean\xf-adobecc2014.exe PE is x32 EXE
21869. <DirEntry 'xgl.dll'>
Z:\training_set\not_classified_clean\xgl.dll PE is x32 DLL
21870. <DirEntry 'XGpuEjectDialog.exe'>
Z:\training_set\not_classified_clean\XGpuEjectDialog.exe PE is x64 EXE
21871. <DirEntry 'ximage3b.dll'>
Z:\training_set\not_classified_clean\ximage3b.dll PE is x32 DLL
21872. <DirEntry 'XI

Z:\training_set\not_classified_clean\xrWPusd.dll PE is x32 DLL
21937. <DirEntry 'xsd (10).exe'>
Z:\training_set\not_classified_clean\xsd (10).exe PE is .NET executable
21938. <DirEntry 'xsd (11).exe'>
Z:\training_set\not_classified_clean\xsd (11).exe PE is .NET executable
21939. <DirEntry 'xsd (12).exe'>
Z:\training_set\not_classified_clean\xsd (12).exe PE is .NET executable
21940. <DirEntry 'xsd (2).exe'>
Z:\training_set\not_classified_clean\xsd (2).exe PE is .NET executable
21941. <DirEntry 'xsd (3).exe'>
Z:\training_set\not_classified_clean\xsd (3).exe PE is .NET executable
21942. <DirEntry 'xsd (4).exe'>
Z:\training_set\not_classified_clean\xsd (4).exe PE is .NET executable
21943. <DirEntry 'xsd (5).exe'>
Z:\training_set\not_classified_clean\xsd (5).exe PE is .NET executable
21944. <DirEntry 'xsd (6).exe'>
Z:\training_set\not_classified_clean\xsd (6).exe PE is .NET executable
21945. <DirEntry 'xsd (8).exe'>
Z:\training_set\not_classified_clean\xsd (8).exe PE is .NET executable
2194